In [1]:
# hello socket
import socket
sk = socket.socket()
print(dir(sk))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_accept', '_check_sendfile_params', '_closed', '_decref_socketios', '_io_refs', '_real_close', '_sendfile_use_send', '_sendfile_use_sendfile', 'accept', 'bind', 'close', 'connect', 'connect_ex', 'detach', 'dup', 'family', 'fileno', 'get_inheritable', 'getblocking', 'getpeername', 'getsockname', 'getsockopt', 'gettimeout', 'ioctl', 'listen', 'makefile', 'proto', 'recv', 'recv_into', 'recvfrom', 'recvfrom_into', 'send', 'sendall', 'sendfile', 'sendto', 'set_inheritable', 'setblocking', 'setsockopt', 'settimeout', 'share', 'shutdown', 'timeout', 'type']


In [ ]:
# socket 常用方法简介
sk = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)

    '''参数一：地址簇
        socket.AF_INET IPv4（默认）
        socket.AF_INET6 IPv6
        socket.AF_UNIX 只能够用于单一的Unix系统进程间通信'''
    
    '''参数二：类型
        socket.SOCK_STREAM　　流式socket , for TCP （默认）
        socket.SOCK_DGRAM　　 数据报式socket , for UDP
        socket.SOCK_RAW 原始套接字，普通的套接字无法处理ICMP、IGMP等网络报文，而
        SOCK_RAW可以；其次，SOCK_RAW也可以处理特殊的IPv4报文；此外，利用原始套接字，可以
        通过IP_HDRINCL套接字选项由用户构造IP头。
        socket.SOCK_RDM 是一种可靠的UDP形式，即保证交付数据报但不保证顺序。
        SOCK_RAM用来提供对原始协议的低级访问，在需要执行某些特殊操作时使用，
        如发送ICMP报文。SOCK_RAM通常仅限于高级用户或管理员运行的程序使用。
        socket.SOCK_SEQPACKET 可靠的连续数据包服务'''
    
    '''参数三：协议
        0　　（默认）与特定的地址家族相关的协议,如果是 0 ，
        则系统就会根据地址格式和套接类别,自动选择一个合适的协议'''

sk.bind(address) # address 格式取决于地址簇，在AF_INET下，以元组（host,port）的形式表示地址

sk.listen(backlog) # backlog 为可缓冲处理的连接队列长度

sk.setblocking(bool) # 是否阻塞（默认True），如果设置False，那么accept和recv时一旦无数据，则报错。

sk.accept() # 接受连接并返回(conn, address),conn是新的套接字对象，用来接收、发送数据，address 为client地址

sk.connect(address) # 同 bind(address)，连接到address处的套接字

sk.connect_ex(address) # 同 connect，不过会有返回值：0-seccess, 10061-fail

sk.close()

sk.recv(bufsize [, flag]) # 一次可以接收的套接字 字节数量
sk.recvfrom(bufsize [, flag]) # 返回值是(data, address),data 同上，address是发送数据的套接字地址

sk.send(string [,flag]) # 将str发送到连接的套接字，返回值是发送的字节数量，如果小于string,可能未将指定内容全部发送
sk.sendall(string [,flag]) # 返回值之前，递归调用send发送所有数据，S：None，F：exception

sk.sendto(string [,flag] ,address) # UDP, 返回值是发送的字节数

sk.settimeout(timeout) #建立套接字时设置的操作超时时间，float/second，None表示未超时

sk.getpeername() # 返回连接套接字的远程地址 (ip, port)
sk.getsockname() # 返回自己的套接字地址 (ip, port)
sk.getaddrinfo(host, port, family=0, type=0, proto=0, flags=0) # 返回远程主机的地址信息
    sk.getaddrinfo('luffycity.com',80)
    sk.getfqdn() # 拿到本机的主机名
    sk.gethostbyname() # 通过域名解析ip地址

sk.fileno() # 套接字的文件描述符


In [ ]:
# socket 通信流程
1.服务器创建 socket: 
    sk.bind(sip, port)

2.服务器socket监听请求(这时候服务器的socket并没有被打开): 
    sk.listen()

3.客户端创建并打开 socket 连接：
    sk.connect(sip, port)

4.服务器socket接收到客户端socket请求，被动打开，开始接收客户端请求，直到客户端返回连接信息
这时候socket进入阻塞状态，所谓阻塞即accept()方法一直等到客户端返回连接信息后才返回，开始接收下一个客户端连接请求
    sk.accept()
    
5.客户端连接成功，向服务器发送连接状态信息：
    sk.send(string)
    
6.服务器accept返回连接成功：
    return sk.accept()

7.客户端向socket写入信息：
    sk.send(string)
    
8.服务器读取信息并回复：
    sk.recv(bufsize) && sk.send(response)
    
9.client.close() && server.close()

'''
TCP是基于链接的，必须先启动服务端，然后再启动客户端去链接服务端

TCP-server
    ss = socket() #创建服务器套接字
    ss.bind()      #把地址绑定到套接字
    ss.listen()      #监听链接
    inf_loop:      #服务器无限循环
        cs = ss.accept() #接受客户端链接
        comm_loop:         #通讯循环
            cs.recv()/cs.send() #对话(接收与发送)
        cs.close()    #关闭客户端套接字
    ss.close()        #关闭服务器套接字(可选)
    
TCP-client
    cs = socket()    # 创建客户套接字
    cs.connect()    # 尝试连接服务器
    comm_loop:        # 通讯循环
        cs.send()/cs.recv()    # 对话(发送/接收)
    cs.close()            # 关闭客户套接字
'''

'''
UDP是无链接的，先启动哪一段都不会报错

UDP-server
    ss = socket()   #创建一个服务器的套接字
    ss.bind()       #绑定服务器套接字
    inf_loop:       #服务器无限循环
        cs = ss.recvfrom()/ss.sendto() # 对话(接收与发送)
     ss.close()                         # 关闭服务器套接字

UDP-client
    cs = socket()   # 创建客户套接字
    comm_loop:      # 通讯循环
        cs.sendto()/cs.recvfrom()   # 对话(发送/接收)
    cs.close()                      # 关闭客户套接字
'''

In [ ]:
# 基于 socket 开发一个聊天程序，实现两端互相发送和接收消息
# tcp-server
import socket
ip_port = ('127.0.0.1', 5656)
sk = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sk.bind(ip_port)
sk.listen(5)
print('server is running')

while True:
    conn, addr = sk.accept()
    data = conn.recv(1024)
    if not data:
        continue
    elif data != b'q':
        print(data.decode('utf-8'))
        conn.send(data.upper())
    else:
        conn.close()
        break
sk.close()

        
# tcp-client
import socket
ip_port = ('127.0.0.1', 5656)

while True:
    sk = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sk.connect(ip_port)
    cmd = input('input your words:').strip()
    sk.send(cmd.encode('utf-8'))
    data = sk.recv(1024)
    data = data.decode('utf-8')
    print(type(data))
    # print('server response:',data)
    if not len(data):
        print('receive data:',data)
    else:
        print('bye')
        sk.close()
        break


In [ ]:
# UDP 实例

# server1
import socket
ip_port=('127.0.0.1',9000)
BUFSIZE=1024
udp_server_client=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
 
udp_server_client.bind(ip_port)
 
while True:
    msg,addr=udp_server_client.recvfrom(BUFSIZE)
    print(msg,addr)
 
    udp_server_client.sendto(msg.upper(),addr)
    
# client1
import socket
ip_port=('127.0.0.1',9000)
BUFSIZE=1024
udp_server_client=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
 
while True:
    msg=input('>>: ').strip()
    if not msg:continue
 
    udp_server_client.sendto(msg.encode('utf-8'),ip_port)
 
    back_msg,addr=udp_server_client.recvfrom(BUFSIZE)
    print(back_msg.decode('utf-8'),addr)
    
    
# server2
from socket import *
from time import strftime
 
ip_port = ('127.0.0.1',9000)
udp_server =socket(AF_INET,SOCK_DGRAM)
udp_server.bind(ip_port)
while True:
    conn,addr  = udp_server.recvfrom(1024)
    print("conn:  ",conn)
    if not conn:
        time_fmt ='%Y-%m-%d %X'
    else:
        time_fmt = conn.decode('utf-8')
    back_msg = strftime(time_fmt)
 
    udp_server.sendto(back_msg.encode('utf-8'),addr)
udp_server.close()

# client2
from socket import *
ip_port = ('127.0.0.1',9000)
udp_client =socket(AF_INET,SOCK_DGRAM)
 
while True:
    msg = input("请输入时间格式(例%Y %m %d)>>: ')").strip()
    udp_client.sendto(msg.encode('utf-8'),ip_port)
    data = udp_client.recv(1024)
    print(data.decode('utf-8'))
udp_client.close()